## Imports

In [1]:
import constants
from constants import *
import importlib
importlib.reload(constants)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from matplotlib import pyplot as plt
import pandas as pd

import time

import sklearn
import tensorflow as tf
import numpy as np
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

#from sklearn.metrics import *
#from sklearn.metrics import classification_report
# import seaborn as sns

import sys


2022-10-21 07:51:01.328364: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-21 07:51:01.459049: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-21 07:51:02.054699: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/lib64/libcudnn.so.8:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/extras/CUPTI/lib64
2022-10-21 07:51:02.054802: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic libr

## Generate tensorflow dataset

Si hay clases muy desbalanceadas, para balancearlas:
1. *Repetir imagenes aleatorias* hasta completar las que faltan para que esté balanceado -> se retrasa el salvado de la red porque los callback no se actualizan hasta que acaba una epoch
2. *Elegir subconjuntos de imagenes aleatorias por cada clase del tamaño de la clase menor y darselas a la red* e ir salvandola (porque la epoch ha terminado) por cada subconjunto que se le pasa

## Generación del dataset

In [2]:
def get_min(dataframe):    
    
    min_class = list(dataframe['class'].value_counts())[-1]
    
    return min_class


def create_pd_dataframes(data_csv, csv_folder = CSV_FOLDER):
    data_panda = pd.read_csv(os.path.join(csv_folder, data_csv))

    # Añadir el path absoluto (la parte desde C:// ... hasta animals al inicio para poder acceder
    data_panda["path"] = [DATASETS_ROOT_FOLDER + item for item in data_panda["path"]]
    
    data_panda["class"] = [CLASS_DICTIONARY[str(item)] for item in data_panda["class"]]
    
    '''
    num_min_instances = get_min(data_panda)
    classes = list(np.unique(data_panda['class']))

    new_dataframe = pd.DataFrame([])

    # new_dataframe.columns = ["class", "path"] #set the header row as the df header

    for clas in classes:
        instances_of_class = data_panda.loc[data_panda["class"] == clas]
        instances_of_class.sample(n = num_min_instances, replace = False)
        new_dataframe = pd.concat([new_dataframe,instances_of_class])
    '''
    return data_panda



def balance_df_classes(dataframe, mini = 0):
    new_dataframe = pd.DataFrame([])

    min_class = get_min(dataframe)

    # Para añadir un numero minimo de imagenes por clase en el caso de meterla por parametros
    if mini > min_class:
        min_class = mini

    # para cada clase coje el numero min de imgs que hay q coger para crear el dataframe
    for clas in list(np.unique(dataframe['class'])):
        current_dataframe = (dataframe[dataframe['class'] == clas])

        aux = current_dataframe.sample(n = min_class, replace = True)

        # y lo concatena con el que ya teniamos
        new_dataframe = pd.concat([new_dataframe,aux])

    return new_dataframe


def create_balanced_dataframes(data_csv, csv_folder = CSV_FOLDER, mini = 0):
    
    pandas_dataframe = create_pd_dataframes(data_csv, csv_folder)
    pandas_dataframe_balanced = balance_df_classes(pandas_dataframe, mini)
    
    return pandas_dataframe_balanced

## Pasar de un dataframe a un dataset de tensorflow

In [3]:
def create_tf_ds(dataframe):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255.
                                  )

    tf_dataset = datagen.flow_from_dataframe(
                            dataframe,
                            x_col = 'path',
                            y_col = 'class',
                            target_size = (IMG_WIDTH,IMG_HEIGHT),
                            color_mode = 'grayscale',
                            batch_size = 32,
                            shuffle = True,
                            interpolation = "bicubic",
                            class_mode='raw' # categorical -> str , raw = numbers
                            )

    return tf_dataset

## Modelo

In [4]:
MODEL_NAME = "test_model"
FROM_LOGITS = True
MODEL_PATH = None

In [5]:
'''if MODEL_PATH:
    model = tf.keras.models.load_model(MODEL_PATH)
else:
    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, N_CHANNELS)),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(CLASS_NUMBER)
    ])'''

model = tf.keras.models.Sequential()
model.add(Conv2D(32,kernel_size=5,padding='same',activation='relu',
        input_shape=(28,28,1)))
model.add(MaxPool2D(padding='same'))
model.add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
model.add(MaxPool2D(padding='same'))
model.add(Conv2D(128,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(padding='same'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(CLASS_NUMBER))

2022-10-21 07:51:02.704564: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-21 07:51:02.704598: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: despachoPC
2022-10-21 07:51:02.704605: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: despachoPC
2022-10-21 07:51:02.704727: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-10-21 07:51:02.704750: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.141.3
2022-10-21 07:51:02.704756: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.141.3
2022-10-21 07:51:02.705043: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructi

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 128)        0

# Hiperparametros del entramiento

In [7]:
# callback to save the best model

# save best model in validation
# monitoriza el accuracy (mejor de validation)
# solo salva el mejor a true
# no solo salva los pesos, salva modelo + pesos en el mismo file
# en validation se salvará el max
# con freq cada epoch
callback_1 = tf.keras.callbacks.ModelCheckpoint(
os.path.join(MODEL_SAVE_FOLDER,MODEL_NAME), monitor='val_sparse_categorical_accuracy', verbose=0, save_best_only=True,
save_weights_only=False, mode='max', save_freq='epoch'
)

# callback to stop the model if it hasnt improved in 50 epochs
callback_2 = tf.keras.callbacks.EarlyStopping(
monitor='val_sparse_categorical_accuracy', min_delta=0.001, patience=50, verbose=0, mode='max',
baseline=None, restore_best_weights=True
)


## Define Parameters (Loss, Accuracy and Optimizer)

In [8]:
# A loss object that receives the raw network output and the one-hot raw class labels is created
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=FROM_LOGITS)

# a metric object that calculates the accuracy is created
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

# optimizer object is created
optimizer = SGD(learning_rate=0.01)

## Compile Model

In [9]:
# compilación del modelo
model.compile(optimizer=optimizer, loss=loss, metrics=accuracy)
# model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

# Bucle de entrenamiento

In [ ]:
for i in range(TRAINING_LOOPS_DATASETS_RENEWAL):
    # TO-DO Poner bonito el bucle para que quede clara la informacion, que no print validated image filenames
    print("\n\n")
    
    ############## DATASET GENERATION ########################
    
    # Pandas dataframe generation from csvs and balanced
    train_df = create_balanced_dataframes(TRAIN_CSV, CSV_FOLDER)   
    val_df = create_balanced_dataframes(VALIDATION_CSV, CSV_FOLDER, mini = 30)
   
    # Tensorflow dataframe generation
    train_tf_dataset = create_tf_ds(train_df)
    validation_tf_dataset = create_tf_ds(val_df)
    
    
    ############### TRAINING PHASE ###################
    
    h = model.fit(train_tf_dataset, epochs= EPOCHS, validation_data = validation_tf_dataset, batch_size = BATCH_NUMBER, callbacks= [callback_1, callback_2], verbose = 1)
    




Found 30350 validated image filenames.
Found 5700 validated image filenames.
Epoch 1/3
946/949 [============================>.] - ETA: 0s - loss: 0.8751 - sparse_categorical_accuracy: 0.7424

INFO:tensorflow:Assets written to: /media/manuel/Extreme SSD/00 IMPORTANTE/Dropbox_Total/Dropbox_Almacen/Estudios/20221009 Alba/Animal_Recognition_Neural_Network-master/models/v5/test_model/assets


INFO:tensorflow:Assets written to: /media/manuel/Extreme SSD/00 IMPORTANTE/Dropbox_Total/Dropbox_Almacen/Estudios/20221009 Alba/Animal_Recognition_Neural_Network-master/models/v5/test_model/assets


949/949 [==============================] - 22s 23ms/step - loss: 0.8731 - sparse_categorical_accuracy: 0.7430 - val_loss: 0.2631 - val_sparse_categorical_accuracy: 0.9240
Epoch 2/3
947/949 [============================>.] - ETA: 0s - loss: 0.1690 - sparse_categorical_accuracy: 0.9484

INFO:tensorflow:Assets written to: /media/manuel/Extreme SSD/00 IMPORTANTE/Dropbox_Total/Dropbox_Almacen/Estudios/20221009 Alba/Animal_Recognition_Neural_Network-master/models/v5/test_model/assets


INFO:tensorflow:Assets written to: /media/manuel/Extreme SSD/00 IMPORTANTE/Dropbox_Total/Dropbox_Almacen/Estudios/20221009 Alba/Animal_Recognition_Neural_Network-master/models/v5/test_model/assets


949/949 [==============================] - 18s 19ms/step - loss: 0.1690 - sparse_categorical_accuracy: 0.9485 - val_loss: 0.1360 - val_sparse_categorical_accuracy: 0.9579
Epoch 3/3
949/949 [==============================] - 17s 18ms/step - loss: 0.1056 - sparse_categorical_accuracy: 0.9674 - val_loss: 0.1501 - val_sparse_categorical_accuracy: 0.9540



Found 30350 validated image filenames.
Found 5700 validated image filenames.
Epoch 1/3
948/949 [============================>.] - ETA: 0s - loss: 0.0907 - sparse_categorical_accuracy: 0.9722

INFO:tensorflow:Assets written to: /media/manuel/Extreme SSD/00 IMPORTANTE/Dropbox_Total/Dropbox_Almacen/Estudios/20221009 Alba/Animal_Recognition_Neural_Network-master/models/v5/test_model/assets


INFO:tensorflow:Assets written to: /media/manuel/Extreme SSD/00 IMPORTANTE/Dropbox_Total/Dropbox_Almacen/Estudios/20221009 Alba/Animal_Recognition_Neural_Network-master/models/v5/test_model/assets


949/949 [==============================] - 19s 20ms/step - loss: 0.0906 - sparse_categorical_accuracy: 0.9722 - val_loss: 0.0866 - val_sparse_categorical_accuracy: 0.9763
Epoch 2/3
421/949 [============>.................] - ETA: 8s - loss: 0.0707 - sparse_categorical_accuracy: 0.9784

In [ ]:
#   with tf.device('/GPU:0'):


In [ ]:
# In case it is decided to train more than 100 epochs
for i in range(epochs):

    with tf.device('/device:GPU:0'):
    #SECUENCIA PARA CONFECCIÓN DEL CSV

    history['epoch'] = list(range(0,len(history.index)))

    history.to_csv(os.path.join(CSV_FOLDER, name_csv_history), header=True, index=False)

    # Plot history: CE
    plt.figure()
    plt.plot(history['epoch'], history['loss'], label='Loss (training data)')
    plt.plot(history['epoch'], history['val_loss'], label='Loss (validation data)')
    plt.title('Loss')
    plt.ylabel('CE')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # Plot history: Accuracy
    plt.figure()
    plt.plot(history['epoch'], history['categorical_accuracy'], label='Accuracy (training data)')
    plt.plot(history['epoch'], history['val_categorical_accuracy'], label='Accuracy (validation data)')
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # show training time
    fin = time.time()
    print(round((fin - inicio)/60, 0))

In [ ]:
# In case it is decided to train more than 100 epochs
for i in range(epochs):

    with tf.device('/device:GPU:0'):
    #SECUENCIA PARA CONFECCIÓN DEL CSV

    history['epoch'] = list(range(0,len(history.index)))

    history.to_csv(os.path.join(CSV_FOLDER, name_csv_history), header=True, index=False)

    # Plot history: CE
    plt.figure()
    plt.plot(history['epoch'], history['loss'], label='Loss (training data)')
    plt.plot(history['epoch'], history['val_loss'], label='Loss (validation data)')
    plt.title('Loss')
    plt.ylabel('CE')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # Plot history: Accuracy
    plt.figure()
    plt.plot(history['epoch'], history['categorical_accuracy'], label='Accuracy (training data)')
    plt.plot(history['epoch'], history['val_categorical_accuracy'], label='Accuracy (validation data)')
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # show training time
    fin = time.time()
    print(round((fin - inicio)/60, 0))